<a href="https://colab.research.google.com/github/ABHIRAM199/MY-ML-Projects/blob/main/LSTM_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Generate a sequence of numbers
sequence = np.array([i for i in range(1, 101)])

# Prepare the data: X = previous 5 numbers, y = next number
def create_dataset(sequence, time_steps):
    X, y = [], []
    for i in range(len(sequence) - time_steps):
        X.append(sequence[i:i + time_steps])
        y.append(sequence[i + time_steps])
    return np.array(X), np.array(y)

In [5]:
# Define time_steps
time_steps = 5

# Create dataset
X, y = create_dataset(sequence, time_steps)

In [4]:
X

array([[ 1,  2,  3,  4,  5],
       [ 2,  3,  4,  5,  6],
       [ 3,  4,  5,  6,  7],
       [ 4,  5,  6,  7,  8],
       [ 5,  6,  7,  8,  9],
       [ 6,  7,  8,  9, 10],
       [ 7,  8,  9, 10, 11],
       [ 8,  9, 10, 11, 12],
       [ 9, 10, 11, 12, 13],
       [10, 11, 12, 13, 14],
       [11, 12, 13, 14, 15],
       [12, 13, 14, 15, 16],
       [13, 14, 15, 16, 17],
       [14, 15, 16, 17, 18],
       [15, 16, 17, 18, 19],
       [16, 17, 18, 19, 20],
       [17, 18, 19, 20, 21],
       [18, 19, 20, 21, 22],
       [19, 20, 21, 22, 23],
       [20, 21, 22, 23, 24],
       [21, 22, 23, 24, 25],
       [22, 23, 24, 25, 26],
       [23, 24, 25, 26, 27],
       [24, 25, 26, 27, 28],
       [25, 26, 27, 28, 29],
       [26, 27, 28, 29, 30],
       [27, 28, 29, 30, 31],
       [28, 29, 30, 31, 32],
       [29, 30, 31, 32, 33],
       [30, 31, 32, 33, 34],
       [31, 32, 33, 34, 35],
       [32, 33, 34, 35, 36],
       [33, 34, 35, 36, 37],
       [34, 35, 36, 37, 38],
       [35, 36

In [6]:
# Reshape X for LSTM input: [samples, time_steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(time_steps, 1)))
model.add(Dense(1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics = ['mse'])

# Train the model
history = model.fit(X, y, epochs=200, verbose=0)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# Test the model
test_input = np.array([96, 97, 98, 99, 100]).reshape((1, time_steps, 1))
predicted = model.predict(test_input, verbose=0)

print(f'Predicted next number in the sequence: {predicted[0][0]}')

Predicted next number in the sequence: 101.04672241210938


In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Example text data
text = """
In the field of Natural Language Processing (NLP), predicting the next word in a sentence is a key task that relies heavily on machine learning models.
These models, particularly Recurrent Neural Networks (RNNs) and Transformers, learn patterns in the structure of language by analyzing vast amounts of
text data. The model is trained to understand the context and the relationship between words, allowing it to generate the next most likely word based
on the given input. One popular architecture used for this task is the GPT (Generative Pretrained Transformer), which is a type of Transformer model.
GPT-based models, such as GPT-3 and GPT-4, have been trained on billions of parameters and have a deep understanding of syntax, semantics, and even
common phrases, allowing them to predict the next word with remarkable accuracy.For example, given the sentence "The cat sat on the", a word prediction
model would predict the next word as "mat," since it has learned that "cat sat on the mat" is a common phrase in the training data.
The process works by encoding the input text into a vector of numbers, which captures the semantic and syntactic information. The model then decodes
this information to predict the most probable next word, considering both the immediate context and larger sentence structures.
By continuously fine-tuning these models and feeding them more diverse datasets, the accuracy and creativity of word prediction in NLP tasks have
improved dramatically.


"""

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [8]:
# Convert text to sequences of integers
input_sequences = []
for line in text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences to the same length
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

In [11]:
# Split data into input (X) and output (y)
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the LSTM model
model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
8/8 - 3s - 323ms/step - accuracy: 0.0517 - loss: 4.8572
Epoch 2/100
8/8 - 0s - 31ms/step - accuracy: 0.0862 - loss: 4.8218
Epoch 3/100
8/8 - 0s - 30ms/step - accuracy: 0.0862 - loss: 4.6888
Epoch 4/100
8/8 - 0s - 37ms/step - accuracy: 0.0862 - loss: 4.5907
Epoch 5/100
8/8 - 0s - 38ms/step - accuracy: 0.0862 - loss: 4.5530
Epoch 6/100
8/8 - 0s - 33ms/step - accuracy: 0.0862 - loss: 4.5259
Epoch 7/100
8/8 - 0s - 35ms/step - accuracy: 0.0862 - loss: 4.5021
Epoch 8/100
8/8 - 0s - 36ms/step - accuracy: 0.0862 - loss: 4.4867
Epoch 9/100
8/8 - 0s - 39ms/step - accuracy: 0.0862 - loss: 4.4611
Epoch 10/100
8/8 - 0s - 35ms/step - accuracy: 0.0862 - loss: 4.4305
Epoch 11/100
8/8 - 0s - 51ms/step - accuracy: 0.0862 - loss: 4.4056
Epoch 12/100
8/8 - 0s - 51ms/step - accuracy: 0.0862 - loss: 4.3608
Epoch 13/100
8/8 - 1s - 76ms/step - accuracy: 0.0862 - loss: 4.3275
Epoch 14/100
8/8 - 1s - 77ms/step - accuracy: 0.0948 - loss: 4.2750
Epoch 15/100
8/8 - 0s - 50ms/step - accuracy: 0.0862 - l

In [12]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text_input, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text_input])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=-1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word

# Test the model
test_text = "prediction"
predicted_word = predict_next_word(model, tokenizer, test_text, max_sequence_len)
print(f'Predicted next word: {predicted_word}')

Predicted next word: model


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.optimizers import Adam

# Sample text data
text = """Success is not final, failure is not fatal: It is the courage to continue that counts." – Winston Churchill"""

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Convert text into sequences of tokens
input_sequences = []
for line in text.split("."):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences to ensure they are of the same length
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Split into input (X) and output (y)
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the LSTM model
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Function to predict the next word

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.1765 - loss: 2.8300
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.1765 - loss: 2.7728
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.1765 - loss: 2.5682
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.1765 - loss: 2.7277
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.2941 - loss: 2.5147
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.2353 - loss: 2.4320
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.1765 - loss: 2.3341
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.1765 - loss: 2.2502
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.2941 - loss: 2.0876
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.2353 - loss: 2.0066
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.2353 - loss: 1.8802
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.4118 - 

In [14]:
def predict_next_word(model, tokenizer, text_sequence, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text_sequence])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    return tokenizer.index_word[predicted[0]]

# Test the model to predict the next word
seed_text = "Winston"
next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_len)
print(f"Predicted next word: {next_word}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Predicted next word: churchill


In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

# Generate a sequence of numbers
sequence = np.array([i for i in range(1, 101)])

# Prepare the data: X = previous 5 numbers, y = next number
def create_dataset(sequence, time_steps):
    X, y = [], []
    for i in range(len(sequence) - time_steps):
        X.append(sequence[i:i + time_steps])
        y.append(sequence[i + time_steps])
    return np.array(X), np.array(y)

# Define time_steps
time_steps = 5

# Create dataset
X, y = create_dataset(sequence, time_steps)

# Reshape X for BiLSTM input: [samples, time_steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Build the BiLSTM model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(time_steps, 1)))
model.add(Dense(1))  # Output layer to predict next number

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=200, verbose=0)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
test_input = np.array([96, 97, 98, 99, 100]).reshape((1, time_steps, 1))
predicted = model.predict(test_input, verbose=0)

print(f'Predicted next number in the sequence: {predicted[0][0]}')

Predicted next number in the sequence: 101.06700134277344


In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

# Example text data
text = """
In the future, AI-powered robots in hospitals could autonomously diagnose patients, assist with surgeries, and provide personalized care.
Using real-time data analysis and machine learning, these robots would adapt to patient needs, optimize treatment plans, and collaborate with
medical staff, transforming healthcare efficiency and precision.

"""

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1  # Add 1 for padding

# Convert text to sequences of integers
input_sequences = []
for line in text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences to the same length
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Split data into input (X) and output (y)
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the BiLSTM model
model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))  # Embedding layer
model.add(Bidirectional(LSTM(100)))  # Bidirectional LSTM layer
model.add(Dense(total_words, activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
2/2 - 4s - 2s/step - accuracy: 0.0222 - loss: 3.7411
Epoch 2/100
2/2 - 0s - 79ms/step - accuracy: 0.0667 - loss: 3.7303
Epoch 3/100
2/2 - 0s - 62ms/step - accuracy: 0.0889 - loss: 3.7224
Epoch 4/100
2/2 - 0s - 68ms/step - accuracy: 0.0889 - loss: 3.7150
Epoch 5/100
2/2 - 0s - 68ms/step - accuracy: 0.0889 - loss: 3.7061
Epoch 6/100
2/2 - 0s - 70ms/step - accuracy: 0.0889 - loss: 3.6962
Epoch 7/100
2/2 - 0s - 70ms/step - accuracy: 0.0889 - loss: 3.6843
Epoch 8/100
2/2 - 0s - 48ms/step - accuracy: 0.0889 - loss: 3.6677
Epoch 9/100
2/2 - 0s - 72ms/step - accuracy: 0.0889 - loss: 3.6427
Epoch 10/100
2/2 - 0s - 66ms/step - accuracy: 0.0889 - loss: 3.6061
Epoch 11/100
2/2 - 0s - 46ms/step - accuracy: 0.1111 - loss: 3.5555
Epoch 12/100
2/2 - 0s - 46ms/step - accuracy: 0.1111 - loss: 3.5355
Epoch 13/100
2/2 - 0s - 70ms/step - accuracy: 0.0889 - loss: 3.4945
Epoch 14/100
2/2 - 0s - 68ms/step - accuracy: 0.0889 - loss: 3.4320
Epoch 15/100
2/2 - 0s - 79ms/step - accuracy: 0.1111 - loss

In [23]:
# Function to predict the next word
def predict_next_word(model, tokenizer, text_input, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text_input])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=-1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word

# Test the model
test_text = "transforming"
predicted_word = predict_next_word(model, tokenizer, test_text, max_sequence_len)
print(f'Predicted next word: {predicted_word}')

Predicted next word: the
